In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_rdd2020_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-16 12:06:20,258 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-16 12:06:20,260 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet18'
        PRETRAINED: True
        #OUT_CHANNELS: [512, 1024, 2048, 1024, 1024, 512] # resnet152
        OUT_CHANNELS: [256, 512, 1024, 1024, 512, 512] # resnet18
        #OUT_CHANNELS: [512, 1024, 512, 512, 256, 256] # model from assignment 4
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [600, 600]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/rdd2020'
DATASET_DIR: "/work/datasets"
EVAL_STEP: 2000 # Evaluate dataset every eval_step, disabled when eval_step < 0
MODEL_SAVE_STEP: 2000 # Save checkpoint every save_step

2021-04-16 12:06:20,261 SSD INFO: Running with config:
DATASETS:
  TEST: ('rdd2020_val',)
  TRAIN: ('rdd2020_train',)
DATASET_DIR: /work/datasets
DATA_LOADER:
  NU

In [ ]:
model = start_train(cfg)

Detector initialized. Total Number of params:  58.19M
Backbone number of parameters: 56.53M
SSD Head number of parameters: 1.66M
2021-04-16 12:06:26,025 SSD.trainer INFO: No checkpoint found.
Dataset loaded. Subset: train, number of images: 8536
2021-04-16 12:06:26,070 SSD.trainer INFO: Start training ...
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
2021-04-16 12:06:37,025 SSD.trainer INFO: iter: 000010, lr: 0.00100, total_loss: 126.793 (126.793), reg_loss: 107.467 (107.467), cls_loss: 19.326 (19.326), time: 0.878 (0.878), eta: 1 day, 5:16:50, mem: 4983M
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Size([16, 512, 1, 1])
torch.Si

In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-03-23 15:00:19,172 SSD INFO: Start evaluating...
2021-03-23 15:00:19,326 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:12<00:00,  7.69it/s]


2021-03-23 15:00:33,086 SSD.inference INFO: mAP: 0.8753
0               : 0.8940
1               : 0.8095
2               : 0.8760
3               : 0.8971
4               : 0.8807
5               : 0.8830
6               : 0.8859
7               : 0.8569
8               : 0.8900
9               : 0.8794



[{'metrics': {'mAP': 0.8752565431542013,
   '0': 0.8939699650469805,
   '1': 0.8094709526574602,
   '2': 0.8760044641851805,
   '3': 0.897146832405082,
   '4': 0.8807326189016644,
   '5': 0.883031870897962,
   '6': 0.8859388314999235,
   '7': 0.8569403048356649,
   '8': 0.8899598842157962,
   '9': 0.8793697068962991}}]